In [2]:
%matplotlib qt
import matplotlib.pyplot as plt #2D plotting library
import matplotlib as mpl
import numpy as np #scientific computing with Python (functions)
import peakutils
from matplotlib import gridspec


In [3]:
data_table = np.genfromtxt('Bwt(AT1)_data.txt').T
data_table.shape

(39, 1691)

In [4]:
#Define the folding time points as name and as value
#labels = np.array(['1h', '30m', '20m', '15m', '10m', '5m', '1m', '40s', '20s', 't0'])
#time = np.array([60*60, 30*60, 20*60, 15*60, 10*60, 5*60, 1*60, 40, 20, 0])[::-1]

labels = np.array(['60m', '30m', '20m', '15m', '10m', '5m', '1m', '40s', '20s', 't0'])
time = np.array([60, 30, 20, 15, 10, 5, 1, 0.66, 0.33, 0])[::-1]

In [5]:
#Remove background with linear equation
data_raw = [np.stack([data_table[0], data_table[2]/100])] + [np.stack([data_table[(4*i)-1], data_table[(4*i)+1]/100]) for i in np.arange(9) + 1]

data_out = []
for d in data_raw:
    x, y = d
    bl = peakutils.baseline(y, 1)

    y_corr = y-bl
    y_corr /= np.trapz(y_corr, x)

    data_out.append([x, y_corr])

f_data = np.array(data_out)[::-1]
f_data.shape

(10, 2, 1691)

In [6]:
#Create timelegend to fill out with complementation
fig, ax = plt.subplots()
for t, (x, y) in zip(time, f_data):
    ax.plot(x, y, label=t)
ax.legend()

In [7]:
time

array([ 0.  ,  0.33,  0.66,  1.  ,  5.  , 10.  , 15.  , 20.  , 30.  ,
       60.  ])

In [8]:
# Check if all the x_arrays are identical
xdata = [x for x, y in f_data]
assert all([np.allclose(x1, x2) for x1, x2 in zip(xdata[1:], xdata[:-1])])
x_size = xdata[0].size
x_size

1691

In [9]:
#Fill out between the timepoints by linear complementation
num_timepoints = 4000
t_axis_lin = np.linspace(0, time.max(), num=num_timepoints)

y_data = f_data[:, 1, :]
output = np.empty((num_timepoints, x_size))

for col_idx, y in enumerate(y_data.T):
    output[:, col_idx] = np.interp(t_axis_lin, time, y)

In [13]:
sclf=0.9
fig, (ax,ax2) = plt.subplots(1,2,sharey=True,figsize=(sclf*5, sclf*4))
ax = plt.subplot2grid((1, 40), (0, 0), colspan=1)
ax2 = plt.subplot2grid((1, 60), (0, 2), colspan=99)


#cmap.set_bad(color='#434343')
#ax.set_yscale('log')

im = ax.imshow(output.T, aspect='auto', interpolation='none', cmap='magma', origin='lower', vmax=0.35,
               extent=[0, time.max(), -117.71,247.93])
im = ax2.imshow(output.T, aspect='auto', interpolation='none', cmap='magma', origin='lower', vmax=0.35,
               extent=[0, time.max(), -117.71,247.93])
ax.set_ylabel('% D uptake')
ax2.set_xlabel('Time (min)')
ax2.set_title('Global HDX PpiB WT 4°C')
ax.set_ylim(0,120)
ax2.set_ylim(0,120)
fig.colorbar(im, label='Fractional population')
#plt.tight_layout()

ax.set_xlim(0, 0.05)
ax2.set_xlim(0.3,20 )
ax.set_yticks([0,20,40,60,80,100])
ax.set_xticks([])
ax2.set_xticks((0.33,5,10,15,20))
ax2.set_yticks([])



plt.savefig('PpiB_linear_20 min_short2.png')

In [10]:
xdata[0].max()

815.989